## Read in data
In this tutorial, it required four input data:

1. "HE_WSI.jpg": High-resolution Whole-slide H&E stained image 
2. "filtered_feature_bc_matrix.h5": The gene expression matrix
3. "tissue_positions_list.csv": Spatial coordinateds
The gene expreesion data can be stored as an AnnData object. AnnData stores a data matrix .X together with annotations of observations .obs and variables .var

The files "HE_WSI.jpg" will be used for ResNet morphological feature extraction, "tissue_positions_list.csv" will be utilized in the image crop process and feature integration, while the file "filtered_feature_bc_matrix.h5" will be used in the gene expression feature matrix module.

# Resnet feature extraction


In [ ]:
path = '../sample/'

In [ ]:
# Import packages
import os
import cv2
import re
import sys
import fileinput

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import skimage.io

img_path=os.path.join(path,'image')
img_ori = cv2.imread(img_path)

if img_ori is None:
    print("Failed to load image")
else:
    print(img_ori.shape)

img = cv2.cvtColor(img_ori, cv2.COLOR_BGR2RGB)
plt.figure(dpi=80)
plt.imshow(img)
plt.show()

The primary stage of image preprocessing involved segmenting whole-slide H&E histological images from patient samples into smaller patches. 
Patches were selected such that each one entirely contained the corresponding spots under the tissue, as indicated in the second column of the 'tissue_positions_list.csv' if it is 1. 
For each spot Si, a corresponding patch was defined, with the center of the patch aligned with the center of the spot, denoted by coordinates (xi, yi) from the last two columns in 'tissue_positions_list.csv'. 
The dimensions of the patch were such that both the height and width were equal to the diameter of the spot, d, as derived from 'scalefactors_json.json'. 
Thus, the boundaries of each patch were determined by the following coordinates: 
the upper boundary at (xi+d/2), the lower boundary at (xi-d/2), the left boundary at (yi-d/2), and the right boundary at (yi + d/2).

In [ ]:
r = 100 #### Need to check diameter EVERYTIME 
for line in open(os.path.join(path, "spatial", "tissue_positions_list.csv"), 'r'):
  if line.startswith('b') or int(line.rstrip().split(',')[1]) == 0:
    continue
  x = int(line.rstrip().split(',')[5])
  y = int(line.rstrip().split(',')[4])
  # print(x)
  # print(y)
  tiles = image_copy[y-r:y+r, x-r:x+r]
  cv2.imwrite(os.path.join(path,'image') +str(x)+'-'+str(y)+'.jpg', tiles)

We utilized a pretrained ResNet-50 model (Tensorflow version: 2.6.0), which was initially trained on the ImageNet dataset, to ensure optimal performance in our task. 
Specifically, we used it with the top fully-connected layer excluded and selected "avg" for the pooling mode to perform feature extraction. 
Apart from these, everything else was set to the default settings.


In [ ]:
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.models import Sequential

resnet = ResNet50(include_top=False, pooling='avg', weights='imagenet', input_shape=(224, 224, 3))
my_new_model = Sequential()
my_new_model.add(resnet)
# Say not to train first layer (ResNet) model. It is already trained
my_new_model.layers[0].trainable = False
my_new_model.summary()

In [ ]:
%%time
from tensorflow.keras.applications.resnet50 import preprocess_input
import cv2 
import numpy as np

resnet_feature_list = []
images = [f for f in os.listdir(DATA_DIR)]

for image in images:
    file = DATA_DIR+image
    
    im = cv2.imread(file)
    im = cv2.resize(im,(224,224))
    img = preprocess_input(np.expand_dims(im.copy(), axis=0))
    resnet_feature = my_new_model.predict(img)
    resnet_feature_np = np.array(resnet_feature)
    resnet_feature_list.append(resnet_feature_np.flatten())

array = np.array(resnet_feature_list)
np.save(os.path.join(path,'image', 'resnet_array.npy'), array) 

In [6]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
array_normalized = scaler.fit_transform(array)

# Gene expression feature exraction

Raw gene expression data underwent Counts Per Million (CPM) normalization, and subsequent log transformation followed by scaling the data to have zero mean and unit variance. All these steps were completed using Scanpy (version:1.6.0). 


In [ ]:
#Read original 10x_h5 data and save it to h5ad
import pandas as pd
import scanpy as sc
from scanpy import read_10x_h5
adata = read_10x_h5(os.path.join(path, "filtered_feature_bc_matrix.h5"))
adata.var_names_make_unique()
spatial=pd.read_csv(os.path.join(path, "spatial", "tissue_positions_list.csv"), sep=",", header=None, na_filter=False,index_col=0) 
adata.obs["x1"]=spatial[1]

#Select captured samples
adata=adata[adata.obs["x1"]==1]
adata.var_names=[i.upper() for i in list(adata.var_names)]
adata.var["genename"]=adata.var.index.astype("str")
adata.write_h5ad(os.path.join(path, "sample.h5ad"))

#Read in gene expression and spatial location
adata=sc.read(os.path.join(path, "sample.h5ad"))

# Perform CPM normalization
sc.pp.normalize_total(adata, target_sum=1e4)
# Log-transform the data
sc.pp.log1p(adata)
sc.pp.scale(adata)

# Resnet morphological and gene expression feature integration

Both the normalized gene expression matrix and morphological feature matrix underwent Principal Component Analysis (PCA), separately processed through the top 10 principal components. The resulting matrices were concatenated based on the corresponding barcode of the spots.

In [24]:
DATA_DIR = os.path.join(path, "image")
images = [f for f in os.listdir(DATA_DIR)]
img_ids_list = [f[0:-4] for f in images]

n = np.zeros((len(img_ids_list), 2))
i = 0
j = 0

for vector in img_ids_list:
  x = int(vector.split('-')[0])
  y = int(vector.split('-')[1])
  n[i][0]=y
  n[i][1]=x
  i = i + 1

image_ids = pd.DataFrame(n)

In [26]:
# Resnet50 PCA
from sklearn.decomposition import PCA

pca = PCA(n_components=10)
resne_fea = pca.fit_transform(array)
resne_fea = pd.DataFrame(resne_fea)

In [27]:
# resnet features with ids
resnet_feature_matrix_combined = pd.concat([image_ids, resne_fea], axis=1, ignore_index=True)
resnet_feature_matrix_combined.rename(columns={0:'imagerow', 1:'imagecol'}, inplace=True )

In [29]:
tissue_position = pd.read_csv((os.path.join(path, "spatial", "tissue_positions_list.csv")), sep=',', names=["barcode", "tissue", "row", "col","imagerow", "imagecol"])
combined_feature_matirx = pd.merge(tissue_position, resnet_feature_matrix_combined, how = "inner", on = ['imagerow', 'imagecol'])
resnet_remix = combined_feature_matirx.drop(["tissue", "row", "col"], axis=1)

resnet_remix.head(5)

,barcode,imagerow,imagecol,2,3,4,5,6,7,8,9,10,11
0,TAAGTAAATGTGCCGC-1,1610,6713,-0.625754,2.208479,-14.621526,-3.867671,-4.253917,2.006824,-1.681317,-7.761388,2.961636,0.106228
1,TGGCACGAGCTCGAGT-1,1609,6831,-1.132644,11.041162,-5.307541,-4.020304,0.948846,-4.475143,-1.311630,-3.712501,6.510522,0.453041
2,ACCGGTCTGAGTACGG-1,1507,6889,-15.165995,-6.775286,-7.886725,-5.320229,-4.165866,3.182706,3.910902,2.281719,2.217093,-1.745553
3,GAACTTAGCGCCCGGT-1,1609,6948,-6.986549,4.449296,-6.655492,-0.533981,-3.192317,-5.359953,0.268802,-4.441774,7.281716,-2.648497
4,AGTAGCTAGACGCCGA-1,1507,7007,-11.896353,-4.481702,-6.624698,-2.810797,-2.488305,-2.267740,5.924125,4.250512,3.352345,-6.920331


In [30]:
# Gene expression in Adata PCA
gene_exp = adata.to_df()
pca = PCA(n_components=10)
embed = pca.fit_transform(gene_exp)

exp_emb = pd.DataFrame(embed)
exp_emb['barcode'] = df.index
exp_emb['barcode'] = exp_emb['barcode'].str.decode('utf8')
barcode = exp_emb["barcode"]
exp_emb = exp_emb.drop(columns=['barcode'])
exp_emb.insert(loc=0, column='barcode', value=barcode)

In [31]:
resnet_plus_exp_PCs = pd.merge(resnet_remix, exp_emb, how = "inner", on = ['barcode']).drop(columns=['imagerow', 'imagecol'])
resnet_plus_exp_PCs.head(5)

,barcode,2_x,3_x,4_x,5_x,6_x,7_x,8_x,9_x,10,...,0,1,2_y,3_y,4_y,5_y,6_y,7_y,8_y,9_y
0,TAAGTAAATGTGCCGC-1,-0.625754,2.208479,-14.621526,-3.867671,-4.253917,2.006824,-1.681317,-7.761388,2.961636,...,29.329668,10.181683,-4.020726,7.109770,1.738399,-5.661458,-0.715062,1.739450,3.392922,-2.633126
1,TGGCACGAGCTCGAGT-1,-1.132644,11.041162,-5.307541,-4.020304,0.948846,-4.475143,-1.311630,-3.712501,6.510522,...,29.114092,4.093781,3.140975,0.607610,-2.490068,-1.254769,-0.112335,-2.849063,-2.263700,-5.127917
2,ACCGGTCTGAGTACGG-1,-15.165995,-6.775286,-7.886725,-5.320229,-4.165866,3.182706,3.910902,2.281719,2.217093,...,32.618317,1.488421,3.387632,-1.013952,-0.396817,-5.833042,-2.133456,0.479996,2.592476,-3.428612
3,GAACTTAGCGCCCGGT-1,-6.986549,4.449296,-6.655492,-0.533981,-3.192317,-5.359953,0.268802,-4.441774,7.281716,...,10.097507,0.669171,-5.001672,-5.451578,-7.546212,4.614475,0.163925,-4.826335,0.888212,-2.120166
4,AGTAGCTAGACGCCGA-1,-11.896353,-4.481702,-6.624698,-2.810797,-2.488305,-2.267740,5.924125,4.250512,3.352345,...,27.485001,4.553941,5.455178,-4.433626,-5.616123,-1.932594,2.287708,-0.626706,0.964270,-3.938417


# Note
Clustering was accomplished using the Louvain modularity optimization algorithm with a resolution range of 1.5-1.9 and k-neighbors set at 39, implemented in Orange Data Mining (version 3.30.1). Make adjustment on resolution and k neighbors to make sure the number of clusters in output equals to the number of clusters from default Louvain clustering on just gene expression (you can find this from either loupe browser or spaceranger /out/analysis/clustering/graphclust/clusters.csv).

In [32]:
for_clustering_input = resnet_plus_exp_PCs.iloc[:,1:]
# save the output from pervious steps to Orange Data Mining
np.savetxt((os.path.join(path, "annotation", "for_clustering_input.csv")), for_clustering_input, delimiter=',')

In [36]:
# Read in the output from 
orange_output = pd.read_csv(os.path.join(path, "annotation", "resnet_plus_exp_clustering_v2.csv"), sep=',', skiprows=3, usecols = [20], header = None)
cluster_label = pd.concat([resnet_plus_exp_PCs, orange_output], axis=1, ignore_index=True)
cluster_label = cluster_label.iloc[:, [0, -1]]
cluster_label.rename( columns={cluster_label.columns[0] :'barcode', cluster_label.columns[-1]:'cluster'}, inplace=True )
cluster_label['cluster'] = cluster_label['cluster'].str.replace(r'\D', '').astype(int)
cluster_label.head(3)

In [39]:
merge = tissue_position.merge(cluster_label, how = 'outer', on = ['barcode'])
louvain_cluster = merge[['barcode', 'cluster']]
merge = merge.astype(float)
louvain_cluster = louvain_cluster.astype(float)
np.savetxt((os.path.join(path, "annotation", "R_E_louvain.csv")), merge, delimiter=',')
np.savetxt((os.path.join(path, "annotation", "R_E_cluster.csv")), louvain_cluster, delimiter=',')
merge.head(3)

,barcode,tissue,row,col,imagerow,imagecol,cluster
0,ACGCCTGACACGCGCT-1,0,0,0,1111,1000,NaN
1,TACCGATCCAACACTT-1,0,1,1,1213,1059,NaN
2,ATTAAAGCGGACGAGC-1,0,0,2,1111,1118,NaN
